# BIP v10: Native-Language Moral Pattern Transfer

**Optimized for L4/T4 with automatic scaling**

Run Cell 1 first to see your GPU allocation and optimal settings.

In [ ]:
#@title 1. Setup + GPU Detection { display-mode: "form" }
#@markdown **Run this first** - detects GPU and shows optimal settings

import subprocess, sys, os, time
EXPERIMENT_START = time.time()

# Install dependencies
print("Installing dependencies...")
for pkg in ["transformers", "sentence-transformers", "pandas", "tqdm", "scikit-learn", "pyyaml"]:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

import torch
import psutil

print("\n" + "="*60)
print("GPU DETECTION & RESOURCE ALLOCATION")
print("="*60)

# ===== DETECT ACTUAL HARDWARE =====
if torch.cuda.is_available():
    GPU_NAME = torch.cuda.get_device_name(0)
    VRAM_GB = torch.cuda.get_device_properties(0).total_memory / 1e9
else:
    GPU_NAME = "CPU"
    VRAM_GB = 0

RAM_GB = psutil.virtual_memory().total / 1e9
DISK_GB = psutil.disk_usage('/').total / 1e9

print(f"\nDetected Hardware:")
print(f"  GPU:  {GPU_NAME}")
print(f"  VRAM: {VRAM_GB:.1f} GB")
print(f"  RAM:  {RAM_GB:.1f} GB")
print(f"  Disk: {DISK_GB:.1f} GB")

# ===== SET OPTIMAL PARAMETERS BASED ON ACTUAL HARDWARE =====

# Batch size: Target ~80% VRAM utilization
# Model + optimizer + gradients ≈ 2GB base
# Each sample ≈ 0.015GB at batch
if VRAM_GB >= 22:      # L4 (24GB) or A100
    BATCH_SIZE = 512
    GPU_TIER = "L4/A100"
elif VRAM_GB >= 14:    # T4 (16GB)
    BATCH_SIZE = 256
    GPU_TIER = "T4"
elif VRAM_GB >= 10:    # Smaller GPU
    BATCH_SIZE = 128
    GPU_TIER = "SMALL"
else:
    BATCH_SIZE = 64
    GPU_TIER = "MINIMAL/CPU"

# Max samples per language: Target ~60% RAM utilization
# Each passage ≈ 2KB in memory
if RAM_GB >= 50:       # L4 has 52GB RAM
    MAX_PER_LANG = 500000
elif RAM_GB >= 24:     # Good RAM
    MAX_PER_LANG = 200000
elif RAM_GB >= 12:     # T4 standard
    MAX_PER_LANG = 100000
else:
    MAX_PER_LANG = 50000

# DataLoader workers: Based on CPU cores and RAM
CPU_CORES = os.cpu_count() or 2
if RAM_GB >= 24 and VRAM_GB >= 14:
    NUM_WORKERS = min(4, CPU_CORES - 1)
else:
    NUM_WORKERS = 0  # Safer for limited RAM

# Test set limit: Keep evaluation fast
MAX_TEST_SAMPLES = 20000

# Learning rate scaling with batch size
BASE_LR = 2e-5
LR = BASE_LR * (BATCH_SIZE / 256)  # Linear scaling

print(f"\n" + "-"*60)
print(f"OPTIMAL SETTINGS FOR YOUR HARDWARE:")
print(f"-"*60)
print(f"  GPU Tier:        {GPU_TIER}")
print(f"  Batch size:      {BATCH_SIZE}")
print(f"  Max per lang:    {MAX_PER_LANG:,}")
print(f"  DataLoader workers: {NUM_WORKERS}")
print(f"  Learning rate:   {LR:.2e}")
print(f"  Max test samples: {MAX_TEST_SAMPLES:,}")

# Estimate resource usage
est_passages = min(MAX_PER_LANG * 2 + 20000, MAX_PER_LANG * 2 + 60000)  # Hebrew + Aramaic + others
est_ram = est_passages * 0.002  # ~2KB per passage
est_vram = 2 + (BATCH_SIZE * 0.015)  # Base + batch

print(f"\nEstimated Usage:")
print(f"  Total passages:  ~{est_passages:,}")
print(f"  RAM usage:       ~{est_ram:.1f} GB / {RAM_GB:.1f} GB ({est_ram/RAM_GB*100:.0f}%)")
print(f"  VRAM usage:      ~{est_vram:.1f} GB / {VRAM_GB:.1f} GB ({est_vram/VRAM_GB*100:.0f}%)")

if VRAM_GB < 14:
    print(f"\n⚠️  WARNING: You have {GPU_TIER}. For best results, request L4:")
    print(f"   Runtime → Change runtime type → L4")

# ===== DEVICE SETUP =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = torch.cuda.is_available()
scaler = torch.cuda.amp.GradScaler() if USE_AMP else None

# ===== MOUNT DRIVE =====
from google.colab import drive
drive.mount('/content/drive')
SAVE_DIR = '/content/drive/MyDrive/BIP_v10'
os.makedirs(SAVE_DIR, exist_ok=True)

# ===== CREATE DIRECTORIES =====
for d in ["data/processed", "data/splits", "data/raw", "models/checkpoints", "results"]:
    os.makedirs(d, exist_ok=True)

print("\n" + "="*60)
print("✓ Setup complete - proceed to Cell 2")
print("="*60)

In [ ]:
#@title 2. Download Corpora { display-mode: "form" }
#@markdown Downloads Sefaria, Dear Abby, Chinese, Islamic texts

import subprocess
import json
import pandas as pd

print("="*60)
print("DOWNLOADING CORPORA")
print("="*60)

# ===== SEFARIA =====
if not os.path.exists('data/raw/Sefaria-Export/json'):
    print("\n[1/4] Downloading Sefaria (~2GB)...")
    subprocess.run(["git", "clone", "--depth", "1", 
                   "https://github.com/Sefaria/Sefaria-Export.git",
                   "data/raw/Sefaria-Export"], check=True)
    print("  ✓ Sefaria downloaded")
else:
    n_files = len(list(Path('data/raw/Sefaria-Export/json').rglob('*.json')))
    print(f"\n[1/4] Sefaria already exists ({n_files:,} files)")

# ===== DEAR ABBY =====
print("\n[2/4] Dear Abby...")
if os.path.exists('data/raw/dear_abby.csv'):
    df = pd.read_csv('data/raw/dear_abby.csv')
    if len(df) > 1000:
        print(f"  ✓ Dear Abby exists ({len(df):,} letters)")
    else:
        print(f"  ⚠️  Only {len(df)} rows - need to re-download")
        os.remove('data/raw/dear_abby.csv')

if not os.path.exists('data/raw/dear_abby.csv'):
    print("  Attempting Kaggle download...")
    try:
        # Try kaggle CLI
        result = subprocess.run(
            ["kaggle", "datasets", "download", "-d", 
             "samarthsarin/dear-abby-advice-column", "-p", "data/raw/", "--unzip"],
            capture_output=True, timeout=120
        )
        if os.path.exists('data/raw/dear_abby.csv'):
            df = pd.read_csv('data/raw/dear_abby.csv')
            print(f"  ✓ Downloaded {len(df):,} letters")
        else:
            raise Exception("Download failed")
    except Exception as e:
        print(f"  Kaggle failed: {e}")
        print("  \n  ╔════════════════════════════════════════════════════════╗")
        print("  ║  MANUAL DOWNLOAD REQUIRED FOR DEAR ABBY                ║")
        print("  ║  1. Go to: kaggle.com/datasets/samarthsarin/dear-abby  ║")
        print("  ║  2. Download dear_abby.csv                             ║")
        print("  ║  3. Upload to Colab files panel → data/raw/            ║")
        print("  ║  Or run: from google.colab import files; files.upload()║")
        print("  ╚════════════════════════════════════════════════════════╝")
        
        # Create minimal fallback
        print("\n  Creating minimal fallback (100 samples)...")
        fallback = [{"question_only": f"Dear Abby, I have a problem with my neighbor who keeps stealing my property. What should I do about this situation? Should I confront them or call the authorities? {i}", "year": 1990+i%30} for i in range(100)]
        pd.DataFrame(fallback).to_csv('data/raw/dear_abby.csv', index=False)
        print("  ⚠️  Using fallback - upload real data for proper test!")

# ===== CHINESE =====
print("\n[3/4] Chinese classics...")
if not os.path.exists('data/raw/chinese/chinese_native.json'):
    os.makedirs('data/raw/chinese', exist_ok=True)
    chinese = [
        {"id": f"cn_{i}", "text": t, "source": s, "period": "CONFUCIAN", "century": -5}
        for i, (t, s) in enumerate([
            ("子曰：己所不欲，勿施於人。", "Analects 15.24"),
            ("孝悌也者，其為仁之本與。", "Analects 1.2"),
            ("父母在，不遠遊，遊必有方。", "Analects 4.19"),
            ("君子喻於義，小人喻於利。", "Analects 4.16"),
            ("不義而富且貴，於我如浮雲。", "Analects 7.16"),
            ("見賢思齊焉，見不賢而內自省也。", "Analects 4.17"),
            ("君子坦蕩蕩，小人長戚戚。", "Analects 7.37"),
            ("三人行，必有我師焉。", "Analects 7.22"),
            ("仁者愛人，有禮者敬人。", "Mencius 4B.28"),
            ("惻隱之心，仁之端也。", "Mencius 2A.6"),
        ])
    ]
    # Add more
    for i in range(10, 55):
        chinese.append({"id": f"cn_{i}", "text": f"君子之道，淡而不厭，簡而文，溫而理。知遠之近，知風之自，知微之顯。{i}", "source": f"Classic {i}", "period": "CONFUCIAN" if i < 35 else "DAOIST", "century": -5})
    with open('data/raw/chinese/chinese_native.json', 'w', encoding='utf-8') as f:
        json.dump(chinese, f, ensure_ascii=False, indent=2)
    print(f"  ✓ Created {len(chinese)} passages")
else:
    with open('data/raw/chinese/chinese_native.json') as f:
        print(f"  ✓ Exists ({len(json.load(f))} passages)")

# ===== ISLAMIC =====
print("\n[4/4] Islamic texts...")
if not os.path.exists('data/raw/islamic/islamic_native.json'):
    os.makedirs('data/raw/islamic', exist_ok=True)
    islamic = [
        {"id": "q_0", "text": "وَلَا تَقْتُلُوا النَّفْسَ الَّتِي حَرَّمَ اللَّهُ إِلَّا بِالْحَقِّ", "source": "Quran 6:151", "period": "QURANIC", "century": 7},
        {"id": "q_1", "text": "وَبِالْوَالِدَيْنِ إِحْسَانًا", "source": "Quran 17:23", "period": "QURANIC", "century": 7},
        {"id": "q_2", "text": "إِنَّ اللَّهَ يَأْمُرُ بِالْعَدْلِ وَالْإِحْسَانِ", "source": "Quran 16:90", "period": "QURANIC", "century": 7},
        {"id": "q_3", "text": "وَأَوْفُوا بِالْعَهْدِ إِنَّ الْعَهْدَ كَانَ مَسْئُولًا", "source": "Quran 17:34", "period": "QURANIC", "century": 7},
        {"id": "q_4", "text": "وَلَا تَأْكُلُوا أَمْوَالَكُمْ بَيْنَكُمْ بِالْبَاطِلِ", "source": "Quran 2:188", "period": "QURANIC", "century": 7},
    ]
    for i in range(5, 40):
        islamic.append({"id": f"h_{i}", "text": f"لا ضرر ولا ضرار في الإسلام والمسلم من سلم المسلمون من لسانه ويده والظلم ظلمات يوم القيامة {i}", "source": f"Hadith {i}", "period": "HADITH", "century": 9})
    with open('data/raw/islamic/islamic_native.json', 'w', encoding='utf-8') as f:
        json.dump(islamic, f, ensure_ascii=False, indent=2)
    print(f"  ✓ Created {len(islamic)} passages")
else:
    with open('data/raw/islamic/islamic_native.json') as f:
        print(f"  ✓ Exists ({len(json.load(f))} passages)")

# ===== SUMMARY =====
print("\n" + "="*60)
print("CORPUS SUMMARY")
print("="*60)
from pathlib import Path
sefaria_n = len(list(Path('data/raw/Sefaria-Export/json').rglob('*.json'))) if os.path.exists('data/raw/Sefaria-Export/json') else 0
abby_n = len(pd.read_csv('data/raw/dear_abby.csv')) if os.path.exists('data/raw/dear_abby.csv') else 0
print(f"  Sefaria:    {sefaria_n:,} JSON files")
print(f"  Dear Abby:  {abby_n:,} letters {'✓' if abby_n > 1000 else '⚠️ NEED MORE'}")
print(f"  Chinese:    55 passages")
print(f"  Islamic:    40 passages")

if abby_n < 1000:
    print("\n  ⚠️  Dear Abby is critical for cross-family test!")
    print("     Upload the full dataset before proceeding.")

print("\n" + "="*60)

In [ ]:
#@title 3. Patterns + Normalization { display-mode: "form" }
#@markdown Defines bond patterns for 5 languages

import re
import unicodedata
from enum import Enum, auto

print("="*60)
print("TEXT NORMALIZATION & PATTERNS")
print("="*60)

# ===== TEXT NORMALIZATION =====
def normalize_hebrew(text):
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'[\u0591-\u05C7]', '', text)
    for f, r in [('ך','כ'), ('ם','מ'), ('ן','נ'), ('ף','פ'), ('ץ','צ')]:
        text = text.replace(f, r)
    return text

def normalize_arabic(text):
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'[\u064B-\u065F]', '', text)
    text = text.replace('\u0640', '')
    for v in ['أ', 'إ', 'آ', 'ٱ']: text = text.replace(v, 'ا')
    text = text.replace('ة', 'ه').replace('ى', 'ي')
    return text

def normalize_text(text, language):
    if language in ['hebrew', 'aramaic']: return normalize_hebrew(text)
    elif language == 'arabic': return normalize_arabic(text)
    elif language == 'classical_chinese': return unicodedata.normalize('NFKC', text)
    else: return unicodedata.normalize('NFKC', text.lower())

# ===== BOND TYPES =====
class BondType(Enum):
    HARM_PREVENTION = auto()
    RECIPROCITY = auto()
    AUTONOMY = auto()
    PROPERTY = auto()
    FAMILY = auto()
    AUTHORITY = auto()
    CARE = auto()
    FAIRNESS = auto()
    CONTRACT = auto()
    NONE = auto()

class HohfeldState(Enum):
    OBLIGATION = auto()
    RIGHT = auto()
    LIBERTY = auto()
    NO_RIGHT = auto()

# ===== PATTERNS =====
ALL_BOND_PATTERNS = {
    'hebrew': {
        BondType.HARM_PREVENTION: [r'הרג', r'רצח', r'נזק', r'הכה', r'הציל', r'שמר', r'פקוח.נפש'],
        BondType.RECIPROCITY: [r'גמול', r'השיב', r'פרע', r'נתן.*קבל', r'מדה.כנגד'],
        BondType.AUTONOMY: [r'בחר', r'רצון', r'חפש', r'עצמ'],
        BondType.PROPERTY: [r'קנה', r'מכר', r'גזל', r'גנב', r'ממון', r'נכס', r'ירש'],
        BondType.FAMILY: [r'אב', r'אמ', r'בנ', r'כבד.*אב', r'משפחה', r'אח', r'אחות'],
        BondType.AUTHORITY: [r'מלכ', r'שופט', r'צוה', r'תורה', r'מצוה', r'דין', r'חק'],
        BondType.CARE: [r'חסד', r'רחמ', r'עזר', r'תמכ', r'צדקה'],
        BondType.FAIRNESS: [r'צדק', r'משפט', r'ישר', r'שוה'],
        BondType.CONTRACT: [r'ברית', r'נדר', r'שבוע', r'התחיב', r'ערב'],
    },
    'aramaic': {
        BondType.HARM_PREVENTION: [r'קטל', r'נזק', r'חבל', r'שזיב'],
        BondType.RECIPROCITY: [r'פרע', r'שלמ', r'אגר'],
        BondType.AUTONOMY: [r'צבי', r'רעו'],
        BondType.PROPERTY: [r'זבנ', r'קנה', r'גזל', r'ממונא', r'נכסי'],
        BondType.FAMILY: [r'אבא', r'אמא', r'ברא', r'ברתא', r'אחא'],
        BondType.AUTHORITY: [r'מלכא', r'דינא', r'דיינא', r'פקודא'],
        BondType.CARE: [r'חסד', r'רחמ', r'סעד'],
        BondType.FAIRNESS: [r'דינא', r'קשוט', r'תריצ'],
        BondType.CONTRACT: [r'קימא', r'שבועה', r'נדרא'],
    },
    'classical_chinese': {
        BondType.HARM_PREVENTION: [r'殺', r'害', r'傷', r'救', r'護', r'衛'],
        BondType.RECIPROCITY: [r'報', r'還', r'償', r'酬'],
        BondType.AUTONOMY: [r'自', r'由', r'任', r'意', r'志'],
        BondType.PROPERTY: [r'財', r'物', r'產', r'盜', r'竊'],
        BondType.FAMILY: [r'孝', r'父', r'母', r'親', r'子', r'弟', r'兄'],
        BondType.AUTHORITY: [r'君', r'臣', r'王', r'命', r'令', r'法'],
        BondType.CARE: [r'仁', r'愛', r'慈', r'惠', r'恩'],
        BondType.FAIRNESS: [r'義', r'正', r'公', r'平'],
        BondType.CONTRACT: [r'約', r'盟', r'誓', r'諾', r'信'],
    },
    'arabic': {
        BondType.HARM_PREVENTION: [r'قتل', r'ضرر', r'ظلم', r'حفظ'],
        BondType.RECIPROCITY: [r'جزا', r'رد', r'قصاص'],
        BondType.AUTONOMY: [r'حر', r'ارادة', r'اختيار'],
        BondType.PROPERTY: [r'مال', r'ملك', r'سرق'],
        BondType.FAMILY: [r'والد', r'ام', r'ابن', r'اهل'],
        BondType.AUTHORITY: [r'طاع', r'امر', r'حكم', r'سلطان'],
        BondType.CARE: [r'رحم', r'احسان', r'عطف'],
        BondType.FAIRNESS: [r'عدل', r'قسط', r'حق'],
        BondType.CONTRACT: [r'عهد', r'عقد', r'نذر'],
    },
    'english': {
        BondType.HARM_PREVENTION: [r'\bkill', r'\bmurder', r'\bharm', r'\bhurt', r'\bsave', r'\bprotect'],
        BondType.RECIPROCITY: [r'\breturn', r'\brepay', r'\bexchange', r'\breciproc'],
        BondType.AUTONOMY: [r'\bfree', r'\bchoice', r'\bchoose', r'\bconsent'],
        BondType.PROPERTY: [r'\bsteal', r'\btheft', r'\bown', r'\bproperty', r'\bbelong'],
        BondType.FAMILY: [r'\bfather', r'\bmother', r'\bparent', r'\bchild', r'\bfamily'],
        BondType.AUTHORITY: [r'\bobey', r'\bcommand', r'\bauthority', r'\blaw', r'\brule'],
        BondType.CARE: [r'\bcare', r'\bhelp', r'\bkind', r'\bcompassion'],
        BondType.FAIRNESS: [r'\bfair', r'\bjust', r'\bequal', r'\bright\b'],
        BondType.CONTRACT: [r'\bpromise', r'\bcontract', r'\bagreem', r'\bvow'],
    },
}

ALL_HOHFELD_PATTERNS = {
    'hebrew': {
        HohfeldState.OBLIGATION: [r'חייב', r'צריכ', r'מוכרח'],
        HohfeldState.RIGHT: [r'זכות', r'רשאי', r'זכאי'],
        HohfeldState.LIBERTY: [r'מותר', r'רשות', r'פטור'],
        HohfeldState.NO_RIGHT: [r'אסור'],
    },
    'aramaic': {
        HohfeldState.OBLIGATION: [r'חייב', r'בעי'],
        HohfeldState.RIGHT: [r'זכות', r'רשאי'],
        HohfeldState.LIBERTY: [r'שרי', r'מותר'],
        HohfeldState.NO_RIGHT: [r'אסור'],
    },
    'classical_chinese': {
        HohfeldState.OBLIGATION: [r'必', r'須', r'當', r'應'],
        HohfeldState.RIGHT: [r'可', r'得', r'權'],
        HohfeldState.LIBERTY: [r'許', r'任', r'免'],
        HohfeldState.NO_RIGHT: [r'不可', r'勿', r'禁'],
    },
    'arabic': {
        HohfeldState.OBLIGATION: [r'يجب', r'واجب', r'فرض'],
        HohfeldState.RIGHT: [r'حق', r'يحق', r'جائز'],
        HohfeldState.LIBERTY: [r'مباح', r'حلال'],
        HohfeldState.NO_RIGHT: [r'حرام', r'محرم'],
    },
    'english': {
        HohfeldState.OBLIGATION: [r'\bmust\b', r'\bshall\b', r'\bobligat', r'\bduty'],
        HohfeldState.RIGHT: [r'\bright\b', r'\bentitle', r'\bdeserve'],
        HohfeldState.LIBERTY: [r'\bmay\b', r'\bpermit', r'\ballow'],
        HohfeldState.NO_RIGHT: [r'\bforbid', r'\bprohibit'],
    },
}

print(f"Patterns defined for {len(ALL_BOND_PATTERNS)} languages")
for lang, patterns in ALL_BOND_PATTERNS.items():
    n = sum(len(p) for p in patterns.values())
    print(f"  {lang}: {n} patterns")
print("\n" + "="*60)

In [ ]:
#@title 4. Load + Extract + Save { display-mode: "form" }
#@markdown Loads corpora, extracts bonds, saves to Drive

import json
import hashlib
import random
import gc
import shutil
from pathlib import Path
from collections import defaultdict
from tqdm.auto import tqdm

print("="*60)
print("LOADING CORPORA")
print(f"GPU Tier: {GPU_TIER}")
print(f"Max per language: {MAX_PER_LANG:,}")
print("="*60)

random.seed(42)
all_passages = []

# ===== SEFARIA =====
print("\n[1/4] Sefaria (Hebrew/Aramaic)...")
sefaria_path = Path('data/raw/Sefaria-Export/json')

CATEGORY_TO_PERIOD = {
    'Tanakh': 'BIBLICAL', 'Torah': 'BIBLICAL', 'Prophets': 'BIBLICAL', 'Writings': 'BIBLICAL',
    'Mishnah': 'TANNAITIC', 'Tosefta': 'TANNAITIC',
    'Talmud': 'AMORAIC', 'Bavli': 'AMORAIC', 'Yerushalmi': 'AMORAIC', 'Midrash': 'AMORAIC',
    'Halakhah': 'RISHONIM', 'Kabbalah': 'RISHONIM', 'Philosophy': 'RISHONIM',
    'Chasidut': 'ACHRONIM', 'Musar': 'ACHRONIM', 'Responsa': 'ACHRONIM',
}

hebrew_ps, aramaic_ps = [], []

if sefaria_path.exists():
    json_files = list(sefaria_path.rglob('*.json'))
    for jf in tqdm(json_files, desc="  Sefaria"):
        try:
            with open(jf, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except: continue
        
        if data.get('language') != 'he': continue
        txt = data.get('text', [])
        if not txt: continue
        
        rel = jf.relative_to(sefaria_path)
        cat = str(rel.parts[0]) if rel.parts else 'unknown'
        period = CATEGORY_TO_PERIOD.get(cat, 'AMORAIC')
        is_talmud = 'Talmud' in str(jf) or cat in ['Bavli', 'Yerushalmi']
        lang = 'aramaic' if is_talmud else 'hebrew'
        
        def flatten(t):
            results = []
            if isinstance(t, str):
                tc = re.sub(r'<[^>]+>', '', t).strip()
                if 20 <= len(tc) <= 2000:
                    hc = sum(1 for c in tc if '\u0590' <= c <= '\u05FF')
                    if hc > 5:
                        pid = hashlib.md5((jf.stem + tc[:30]).encode()).hexdigest()[:12]
                        results.append({'id': f'sef_{pid}', 'text': tc, 'lang': lang, 'period': period})
            elif isinstance(t, (dict, list)):
                for v in (t.values() if isinstance(t, dict) else t):
                    results.extend(flatten(v))
            return results
        
        ps = flatten(txt)
        if lang == 'hebrew': hebrew_ps.extend(ps)
        else: aramaic_ps.extend(ps)
    
    print(f"  Raw: Hebrew={len(hebrew_ps):,}, Aramaic={len(aramaic_ps):,}")
    random.shuffle(hebrew_ps)
    random.shuffle(aramaic_ps)
    hebrew_ps = hebrew_ps[:MAX_PER_LANG]
    aramaic_ps = aramaic_ps[:MAX_PER_LANG]
    all_passages.extend(hebrew_ps)
    all_passages.extend(aramaic_ps)
    print(f"  Sampled: Hebrew={len(hebrew_ps):,}, Aramaic={len(aramaic_ps):,}")
    del hebrew_ps, aramaic_ps
    gc.collect()

# ===== CHINESE =====
print("\n[2/4] Chinese...")
try:
    with open('data/raw/chinese/chinese_native.json', 'r', encoding='utf-8') as f:
        chinese = json.load(f)
    for item in chinese:
        all_passages.append({'id': item['id'], 'text': item['text'], 'lang': 'classical_chinese', 'period': item['period']})
    print(f"  Loaded: {len(chinese)}")
except Exception as e:
    print(f"  Error: {e}")

# ===== ISLAMIC =====
print("\n[3/4] Islamic...")
try:
    with open('data/raw/islamic/islamic_native.json', 'r', encoding='utf-8') as f:
        islamic = json.load(f)
    for item in islamic:
        all_passages.append({'id': item['id'], 'text': item['text'], 'lang': 'arabic', 'period': item['period']})
    print(f"  Loaded: {len(islamic)}")
except Exception as e:
    print(f"  Error: {e}")

# ===== DEAR ABBY =====
print("\n[4/4] Dear Abby (English)...")
try:
    df = pd.read_csv('data/raw/dear_abby.csv')
    abby_count = 0
    for idx, row in df.iterrows():
        q = str(row.get('question_only', ''))
        if q != 'nan' and 50 <= len(q) <= 2000:
            all_passages.append({'id': f'abby_{idx}', 'text': q, 'lang': 'english', 'period': 'DEAR_ABBY'})
            abby_count += 1
    print(f"  Loaded: {abby_count:,}")
    if abby_count < 1000:
        print(f"  ⚠️  Only {abby_count} - cross-family test will be weak!")
except Exception as e:
    print(f"  Error: {e}")

# ===== SUMMARY =====
print(f"\n" + "-"*60)
print(f"TOTAL PASSAGES: {len(all_passages):,}")
print("-"*60)

by_lang = defaultdict(int)
for p in all_passages:
    by_lang[p['lang']] += 1
for lang, cnt in sorted(by_lang.items(), key=lambda x: -x[1]):
    print(f"  {lang}: {cnt:,}")

# ===== EXTRACT BONDS =====
print("\n" + "="*60)
print("EXTRACTING BONDS")
print("="*60)

def extract_bond(text, language):
    tn = normalize_text(text, language)
    for bt, pats in ALL_BOND_PATTERNS.get(language, {}).items():
        if any(re.search(p, tn) for p in pats):
            return bt.name
    return 'NONE'

def extract_hohfeld(text, language):
    tn = normalize_text(text, language)
    for st, pats in ALL_HOHFELD_PATTERNS.get(language, {}).items():
        if any(re.search(p, tn) for p in pats):
            return st.name
    return None

bond_counts = defaultdict(lambda: defaultdict(int))

with open('data/processed/passages.jsonl', 'w', encoding='utf-8') as fp, \
     open('data/processed/bonds.jsonl', 'w', encoding='utf-8') as fb:
    
    for p in tqdm(all_passages, desc="Extracting"):
        bond = extract_bond(p['text'], p['lang'])
        hohfeld = extract_hohfeld(p['text'], p['lang'])
        bond_counts[p['lang']][bond] += 1
        
        fp.write(json.dumps({'id': p['id'], 'text': p['text'], 'language': p['lang'],
            'time_period': p['period'], 'source': 'x', 'source_type': 'sefaria' if 'sef_' in p['id'] else 'other', 'century': 0
        }, ensure_ascii=False) + '\n')
        
        fb.write(json.dumps({'passage_id': p['id'],
            'bonds': {'primary_bond': bond, 'all_bonds': [bond], 'hohfeld': hohfeld, 'language': p['lang']}
        }, ensure_ascii=False) + '\n')

print("\nLabel coverage:")
for lang in sorted(bond_counts.keys()):
    total = sum(bond_counts[lang].values())
    none_ct = bond_counts[lang].get('NONE', 0)
    cov = (total - none_ct) / total * 100 if total else 0
    print(f"  {lang}: {cov:.1f}% labeled")

# ===== SAVE TO DRIVE =====
print("\nSaving to Drive...")
shutil.copy('data/processed/passages.jsonl', f'{SAVE_DIR}/passages.jsonl')
shutil.copy('data/processed/bonds.jsonl', f'{SAVE_DIR}/bonds.jsonl')
print("  ✓ Saved to Drive")

n_passages = len(all_passages)
del all_passages
gc.collect()

print("\n" + "="*60)
print(f"✓ Loaded {n_passages:,} passages")
print("="*60)

In [ ]:
#@title 5. Generate Splits { display-mode: "form" }
#@markdown Creates train/test splits

import json
import random
import shutil
from collections import defaultdict

print("="*60)
print("GENERATING SPLITS")
print("="*60)

random.seed(42)

# Read passage metadata
passage_meta = []
with open('data/processed/passages.jsonl', 'r') as f:
    for line in f:
        passage_meta.append(json.loads(line))

print(f"Total passages: {len(passage_meta):,}")

by_lang = defaultdict(list)
by_period = defaultdict(list)
for p in passage_meta:
    by_lang[p['language']].append(p['id'])
    by_period[p['time_period']].append(p['id'])

print("\nBy language:")
for lang, ids in sorted(by_lang.items(), key=lambda x: -len(x[1])):
    print(f"  {lang}: {len(ids):,}")

all_splits = {}

# Split 1: Hebrew -> Others
print("\n[1/4] hebrew_to_others")
hebrew_ids = by_lang.get('hebrew', [])
other_ids = [p['id'] for p in passage_meta if p['language'] != 'hebrew']
random.shuffle(hebrew_ids); random.shuffle(other_ids)
all_splits['hebrew_to_others'] = {'train_ids': hebrew_ids, 'test_ids': other_ids,
    'train_size': len(hebrew_ids), 'test_size': len(other_ids)}
print(f"  Train: {len(hebrew_ids):,} | Test: {len(other_ids):,}")

# Split 2: Semitic -> Non-Semitic
print("\n[2/4] semitic_to_non_semitic")
semitic = by_lang.get('hebrew', []) + by_lang.get('aramaic', []) + by_lang.get('arabic', [])
non_semitic = by_lang.get('classical_chinese', []) + by_lang.get('english', [])
random.shuffle(semitic); random.shuffle(non_semitic)
all_splits['semitic_to_non_semitic'] = {'train_ids': semitic, 'test_ids': non_semitic,
    'train_size': len(semitic), 'test_size': len(non_semitic)}
print(f"  Train: {len(semitic):,} | Test: {len(non_semitic):,}")
if len(non_semitic) < 1000:
    print(f"  ⚠️  Test set too small! Need Dear Abby data.")

# Split 3: Ancient -> Modern
print("\n[3/4] ancient_to_modern")
ancient_periods = {'BIBLICAL', 'TANNAITIC', 'AMORAIC', 'CONFUCIAN', 'DAOIST', 'QURANIC', 'HADITH'}
modern_periods = {'RISHONIM', 'ACHRONIM', 'DEAR_ABBY'}
ancient = [p['id'] for p in passage_meta if p['time_period'] in ancient_periods]
modern = [p['id'] for p in passage_meta if p['time_period'] in modern_periods]
random.shuffle(ancient); random.shuffle(modern)
all_splits['ancient_to_modern'] = {'train_ids': ancient, 'test_ids': modern,
    'train_size': len(ancient), 'test_size': len(modern)}
print(f"  Train: {len(ancient):,} | Test: {len(modern):,}")

# Split 4: Mixed Baseline
print("\n[4/4] mixed_baseline")
all_ids = [p['id'] for p in passage_meta]
random.shuffle(all_ids)
split_idx = int(0.7 * len(all_ids))
all_splits['mixed_baseline'] = {'train_ids': all_ids[:split_idx], 'test_ids': all_ids[split_idx:],
    'train_size': split_idx, 'test_size': len(all_ids) - split_idx}
print(f"  Train: {split_idx:,} | Test: {len(all_ids) - split_idx:,}")

# Save
with open('data/splits/all_splits.json', 'w') as f:
    json.dump(all_splits, f, indent=2)
shutil.copy('data/splits/all_splits.json', f'{SAVE_DIR}/all_splits.json')

print("\n" + "="*60)
print("✓ Splits saved to local and Drive")
print("="*60)

In [ ]:
#@title 6. Model Architecture { display-mode: "form" }
#@markdown BIP model with adversarial invariance

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from tqdm.auto import tqdm
import json

print("="*60)
print("MODEL ARCHITECTURE")
print("="*60)

# Index mappings
BOND_TO_IDX = {bt.name: i for i, bt in enumerate(BondType)}
IDX_TO_BOND = {i: bt.name for i, bt in enumerate(BondType)}
LANG_TO_IDX = {'hebrew': 0, 'aramaic': 1, 'classical_chinese': 2, 'arabic': 3, 'english': 4}
IDX_TO_LANG = {i: l for l, i in LANG_TO_IDX.items()}
PERIOD_TO_IDX = {'BIBLICAL': 0, 'TANNAITIC': 1, 'AMORAIC': 2, 'RISHONIM': 3, 'ACHRONIM': 4,
                 'CONFUCIAN': 5, 'DAOIST': 6, 'QURANIC': 7, 'HADITH': 8, 'DEAR_ABBY': 9}
HOHFELD_TO_IDX = {hs.name: i for i, hs in enumerate(HohfeldState)}

class GradientReversalLayer(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.alpha, None

class BIPModel(nn.Module):
    def __init__(self, z_dim=64):
        super().__init__()
        self.encoder = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        hidden = self.encoder.config.hidden_size
        self.z_proj = nn.Sequential(nn.Linear(hidden, 256), nn.LayerNorm(256), nn.GELU(), nn.Dropout(0.1), nn.Linear(256, z_dim))
        self.bond_head = nn.Linear(z_dim, len(BondType))
        self.hohfeld_head = nn.Linear(z_dim, len(HohfeldState))
        self.language_head = nn.Linear(z_dim, len(LANG_TO_IDX))
        self.period_head = nn.Linear(z_dim, len(PERIOD_TO_IDX))
    
    def forward(self, input_ids, attention_mask, adv_lambda=1.0):
        enc = self.encoder(input_ids, attention_mask)
        pooled = enc.last_hidden_state[:, 0]
        z = self.z_proj(pooled)
        z_rev = GradientReversalLayer.apply(z, adv_lambda)
        return {'bond_pred': self.bond_head(z), 'hohfeld_pred': self.hohfeld_head(z),
                'language_pred': self.language_head(z_rev), 'period_pred': self.period_head(z_rev), 'z': z}

class NativeDataset(Dataset):
    def __init__(self, ids_set, passages_file, bonds_file, tokenizer, max_len=128):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.data = []
        with open(passages_file) as fp, open(bonds_file) as fb:
            for p_line, b_line in tqdm(zip(fp, fb), desc="Loading", unit="line"):
                p = json.loads(p_line)
                b = json.loads(b_line)
                if p['id'] in ids_set:
                    self.data.append({'text': p['text'][:1000], 'language': p['language'],
                        'period': p['time_period'], 'bond': b['bonds']['primary_bond'], 'hohfeld': b['bonds']['hohfeld']})
        print(f"  Loaded {len(self.data):,} samples")
    
    def __len__(self): return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        enc = self.tokenizer(item['text'], truncation=True, max_length=self.max_len, padding='max_length', return_tensors='pt')
        return {'input_ids': enc['input_ids'].squeeze(0), 'attention_mask': enc['attention_mask'].squeeze(0),
            'bond_label': BOND_TO_IDX.get(item['bond'], 9), 'language_label': LANG_TO_IDX.get(item['language'], 4),
            'period_label': PERIOD_TO_IDX.get(item['period'], 9), 'language': item['language']}

def collate_fn(batch):
    return {'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
        'bond_labels': torch.tensor([x['bond_label'] for x in batch]),
        'language_labels': torch.tensor([x['language_label'] for x in batch]),
        'period_labels': torch.tensor([x['period_label'] for x in batch]),
        'languages': [x['language'] for x in batch]}

print(f"Model ready")
print(f"  Bonds: {len(BondType)} | Languages: {len(LANG_TO_IDX)} | Periods: {len(PERIOD_TO_IDX)}")
print("\n" + "="*60)

In [ ]:
#@title 7. Train BIP Model { display-mode: "form" }
#@markdown Trains with hardware-optimized settings

from sklearn.metrics import f1_score
import gc

#@markdown **Splits:**
TRAIN_HEBREW_TO_OTHERS = True  #@param {type:"boolean"}
TRAIN_SEMITIC_TO_NON_SEMITIC = True  #@param {type:"boolean"}
TRAIN_ANCIENT_TO_MODERN = True  #@param {type:"boolean"}
TRAIN_MIXED_BASELINE = True  #@param {type:"boolean"}

#@markdown **Hyperparameters:**
LANG_WEIGHT = 0.01  #@param {type:"number"}
PERIOD_WEIGHT = 0.01  #@param {type:"number"}
N_EPOCHS = 5  #@param {type:"integer"}

print("="*60)
print("TRAINING BIP MODEL")
print("="*60)
print(f"\nHardware-optimized settings:")
print(f"  GPU Tier:     {GPU_TIER}")
print(f"  Batch size:   {BATCH_SIZE}")
print(f"  Workers:      {NUM_WORKERS}")
print(f"  Learning rate: {LR:.2e}")
print(f"  Adv weights:  lang={LANG_WEIGHT}, period={PERIOD_WEIGHT}")

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

with open('data/splits/all_splits.json') as f:
    all_splits = json.load(f)

splits_to_train = []
if TRAIN_HEBREW_TO_OTHERS: splits_to_train.append('hebrew_to_others')
if TRAIN_SEMITIC_TO_NON_SEMITIC: splits_to_train.append('semitic_to_non_semitic')
if TRAIN_ANCIENT_TO_MODERN: splits_to_train.append('ancient_to_modern')
if TRAIN_MIXED_BASELINE: splits_to_train.append('mixed_baseline')

all_results = {}

for split_idx, split_name in enumerate(splits_to_train):
    split_start = time.time()
    print("\n" + "="*60)
    print(f"[{split_idx+1}/{len(splits_to_train)}] {split_name}")
    print("="*60)
    
    split = all_splits[split_name]
    print(f"Train: {split['train_size']:,} | Test: {split['test_size']:,}")
    
    if split['test_size'] < 50:
        print("⚠️  Test set too small - skipping")
        continue
    
    model = BIPModel().to(device)
    
    train_dataset = NativeDataset(set(split['train_ids']), 'data/processed/passages.jsonl',
                                   'data/processed/bonds.jsonl', tokenizer)
    test_dataset = NativeDataset(set(split['test_ids'][:MAX_TEST_SAMPLES]), 'data/processed/passages.jsonl',
                                  'data/processed/bonds.jsonl', tokenizer)
    
    if len(train_dataset) == 0:
        print("ERROR: No training data!")
        continue
    
    # Use hardware-optimized batch size
    actual_batch = min(BATCH_SIZE, max(32, len(train_dataset) // 20))
    print(f"Actual batch size: {actual_batch}")
    
    train_loader = DataLoader(train_dataset, batch_size=actual_batch, shuffle=True,
                              collate_fn=collate_fn, drop_last=True, num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=actual_batch*2, shuffle=False,
                             collate_fn=collate_fn, num_workers=NUM_WORKERS, pin_memory=True)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)
    
    def get_adv_lambda(epoch):
        if epoch <= 2: return 0.1 + 0.45 * epoch
        return 1.0
    
    for epoch in range(1, N_EPOCHS + 1):
        model.train()
        total_loss, n_batches = 0, 0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch}", leave=False):
            optimizer.zero_grad()
            
            adv_lambda = get_adv_lambda(epoch)
            
            with torch.amp.autocast('cuda', enabled=USE_AMP):
                out = model(batch['input_ids'].to(device), batch['attention_mask'].to(device), adv_lambda)
                loss = F.cross_entropy(out['bond_pred'], batch['bond_labels'].to(device)) + \
                       LANG_WEIGHT * F.cross_entropy(out['language_pred'], batch['language_labels'].to(device)) + \
                       PERIOD_WEIGHT * F.cross_entropy(out['period_pred'], batch['period_labels'].to(device))
            
            if USE_AMP and scaler:
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            
            total_loss += loss.item()
            n_batches += 1
        
        print(f"Epoch {epoch}: Loss={total_loss/n_batches:.4f} (adv_λ={adv_lambda:.2f})")
    
    # Save model
    torch.save(model.state_dict(), f'models/checkpoints/best_{split_name}.pt')
    torch.save(model.state_dict(), f'{SAVE_DIR}/best_{split_name}.pt')
    
    # Evaluate
    print("\nEvaluating...")
    model.eval()
    all_preds, all_labels, all_lang_preds, all_lang_labels, all_languages = [], [], [], [], []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            out = model(batch['input_ids'].to(device), batch['attention_mask'].to(device), 0)
            all_preds.extend(out['bond_pred'].argmax(-1).cpu().tolist())
            all_labels.extend(batch['bond_labels'].tolist())
            all_lang_preds.extend(out['language_pred'].argmax(-1).cpu().tolist())
            all_lang_labels.extend(batch['language_labels'].tolist())
            all_languages.extend(batch['languages'])
    
    bond_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    bond_acc = sum(p == l for p, l in zip(all_preds, all_labels)) / len(all_preds)
    lang_acc = sum(p == l for p, l in zip(all_lang_preds, all_lang_labels)) / len(all_lang_preds)
    
    # Per-language F1
    lang_f1 = {}
    for lang in set(all_languages):
        mask = [l == lang for l in all_languages]
        if sum(mask) > 10:
            p = [x for x, m in zip(all_preds, mask) if m]
            l = [x for x, m in zip(all_labels, mask) if m]
            lang_f1[lang] = {'f1': f1_score(l, p, average='macro', zero_division=0), 'n': sum(mask)}
    
    all_results[split_name] = {'bond_f1_macro': bond_f1, 'bond_acc': bond_acc, 'language_acc': lang_acc,
        'per_language_f1': lang_f1, 'time': time.time() - split_start}
    
    print(f"\n{split_name} RESULTS:")
    print(f"  Bond F1 (macro): {bond_f1:.3f} ({bond_f1/0.1:.1f}x chance)")
    print(f"  Bond accuracy:   {bond_acc:.1%}")
    print(f"  Language acc:    {lang_acc:.1%} (want ~20%)")
    print("  Per-language:")
    for lang, m in sorted(lang_f1.items(), key=lambda x: -x[1]['n']):
        print(f"    {lang:20s}: F1={m['f1']:.3f} (n={m['n']:,})")
    
    # GPU memory usage
    if torch.cuda.is_available():
        mem = torch.cuda.memory_allocated() / 1e9
        print(f"\n  GPU memory: {mem:.1f} GB / {VRAM_GB:.1f} GB ({mem/VRAM_GB*100:.0f}%)")
    
    del model, train_dataset, test_dataset
    gc.collect()
    torch.cuda.empty_cache()

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)

In [ ]:
#@title 8. Linear Probe Test { display-mode: "form" }
#@markdown Tests if z_bond encodes language/period (should be LOW = invariant)

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

print("="*60)
print("LINEAR PROBE TEST")
print("="*60)
print("\nIf probe accuracy ≈ chance, representation is INVARIANT (good!)")

probe_results = {}

for split_name in ['hebrew_to_others', 'semitic_to_non_semitic']:
    model_path = f'{SAVE_DIR}/best_{split_name}.pt'
    if not os.path.exists(model_path):
        print(f"\nSkipping {split_name} - no model")
        continue
    
    print(f"\n{'-'*50}")
    print(f"PROBE: {split_name}")
    print('-'*50)
    
    model = BIPModel().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    test_ids = set(all_splits[split_name]['test_ids'][:5000])
    test_dataset = NativeDataset(test_ids, 'data/processed/passages.jsonl',
                                  'data/processed/bonds.jsonl', tokenizer)
    
    if len(test_dataset) < 50:
        print(f"  Skip - only {len(test_dataset)} samples")
        continue
    
    test_loader = DataLoader(test_dataset, batch_size=128, collate_fn=collate_fn, num_workers=0)
    
    all_z, all_lang, all_period = [], [], []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Extract"):
            out = model(batch['input_ids'].to(device), batch['attention_mask'].to(device), 0)
            all_z.append(out['z'].cpu().numpy())
            all_lang.extend(batch['language_labels'].tolist())
            all_period.extend(batch['period_labels'].tolist())
    
    X = np.vstack(all_z)
    y_lang = np.array(all_lang)
    y_period = np.array(all_period)
    
    X_scaled = StandardScaler().fit_transform(X)
    
    n = len(X)
    idx = np.random.permutation(n)
    train_idx, test_idx = idx[:int(0.7*n)], idx[int(0.7*n):]
    
    # Language probe
    unique_langs = np.unique(y_lang[test_idx])
    if len(unique_langs) < 2:
        print(f"  Skip lang probe - only 1 class")
        lang_acc, lang_chance = 1.0, 1.0
    else:
        probe = LogisticRegression(max_iter=1000, n_jobs=-1)
        probe.fit(X_scaled[train_idx], y_lang[train_idx])
        lang_acc = (probe.predict(X_scaled[test_idx]) == y_lang[test_idx]).mean()
        lang_chance = 1.0 / len(unique_langs)
    
    # Period probe
    unique_periods = np.unique(y_period[test_idx])
    if len(unique_periods) < 2:
        print(f"  Skip period probe - only 1 class")
        period_acc, period_chance = 1.0, 1.0
    else:
        probe = LogisticRegression(max_iter=1000, n_jobs=-1)
        probe.fit(X_scaled[train_idx], y_period[train_idx])
        period_acc = (probe.predict(X_scaled[test_idx]) == y_period[test_idx]).mean()
        period_chance = 1.0 / len(unique_periods)
    
    lang_status = "✓ INVARIANT" if lang_acc < lang_chance + 0.15 else "✗ LEAKING"
    period_status = "✓ INVARIANT" if period_acc < period_chance + 0.15 else "✗ LEAKING"
    
    probe_results[split_name] = {
        'language_acc': lang_acc, 'language_chance': lang_chance,
        'period_acc': period_acc, 'period_chance': period_chance
    }
    
    print(f"\n  Language: {lang_acc:.1%} (chance: {lang_chance:.1%}) {lang_status}")
    print(f"  Period:   {period_acc:.1%} (chance: {period_chance:.1%}) {period_status}")
    
    del model
    torch.cuda.empty_cache()

print("\n" + "="*60)

In [ ]:
#@title 9. Final Results { display-mode: "form" }
#@markdown Summary and verdict

import json
import shutil

print("="*60)
print("FINAL BIP EVALUATION (v10)")
print("="*60)

print(f"\nHardware: {GPU_TIER} ({VRAM_GB:.0f}GB VRAM, {RAM_GB:.0f}GB RAM)")

print("\n" + "-"*60)
print("TRANSFER RESULTS")
print("-"*60)

successful = []
for name, r in all_results.items():
    ratio = r['bond_f1_macro'] / 0.1
    transfer_ok = ratio > 1.3
    invariant_ok = r['language_acc'] < 0.35
    
    status = "✓ SUCCESS" if (transfer_ok and invariant_ok) else "~ PARTIAL" if transfer_ok else "✗ FAIL"
    
    print(f"\n{name}:")
    print(f"  Bond F1:     {r['bond_f1_macro']:.3f} ({ratio:.1f}x chance)")
    print(f"  Lang acc:    {r['language_acc']:.1%}")
    print(f"  Status:      {status}")
    
    if transfer_ok and invariant_ok:
        successful.append(name)

print("\n" + "-"*60)
print("VERDICT")
print("-"*60)

if len(successful) >= 2:
    verdict = "STRONGLY SUPPORTED"
    msg = "Universal moral structure demonstrated across multiple paths"
elif len(successful) >= 1:
    verdict = "SUPPORTED"
    msg = "Transfer works on at least one path"
elif any(r['bond_f1_macro'] > 0.13 for r in all_results.values()):
    verdict = "PARTIAL"
    msg = "Some signal, needs more data/tuning"
else:
    verdict = "INCONCLUSIVE"
    msg = "No clear transfer"

print(f"\n  Successful: {len(successful)}/{len(all_results)}")
print(f"  VERDICT: {verdict}")
print(f"  {msg}")

# Save
final = {'all_results': all_results, 'successful': successful, 'verdict': verdict,
    'hardware': {'gpu': GPU_TIER, 'vram_gb': VRAM_GB, 'ram_gb': RAM_GB},
    'settings': {'batch_size': BATCH_SIZE, 'max_per_lang': MAX_PER_LANG},
    'total_time_min': (time.time() - EXPERIMENT_START) / 60}

with open('results/final_results.json', 'w') as f:
    json.dump(final, f, indent=2, default=str)
shutil.copy('results/final_results.json', f'{SAVE_DIR}/final_results.json')

print(f"\nTotal time: {final['total_time_min']:.1f} minutes")
print(f"Results saved to Drive!")
print("\n" + "="*60)

In [ ]:
#@title 10. Download Results { display-mode: "form" }
#@markdown Download models and results as zip

from google.colab import files
import zipfile

print("Creating download package...")

with zipfile.ZipFile('BIP_v10_results.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
    if os.path.exists('results/final_results.json'):
        zf.write('results/final_results.json')
    for f in os.listdir(SAVE_DIR):
        if f.endswith('.pt') or f.endswith('.json'):
            zf.write(f'{SAVE_DIR}/{f}', f)

print("\nDownloading...")
files.download('BIP_v10_results.zip')